In [1]:
import math
import time
import numpy as np
from tqdm import tqdm
import torch
import gpytorch
from matplotlib import pyplot as plt
from ucimlrepo import fetch_ucirepo 
from math import floor
from torch.utils.data import TensorDataset, DataLoader


# Make plots inline
%matplotlib inline

In [2]:
def downloader(uci_id):
    # fetch dataset 
    uci_download = fetch_ucirepo(id=uci_id)
    
    # data (as pandas dataframes) 
    X_data = uci_download.data.features
    # Drop category and date variables
    if uci_id == 1:
        X_data = X_data.drop(["Sex"], axis=1)
    if uci_id == 275:
        X_data = X_data.drop(["dteday"], axis=1)
    if uci_id == 374:
        X_data = X_data.drop(["date"], axis=1)
    if uci_id == 183:
        X_data = X_data.drop(["communityname"], axis=1)
        object_columns = X_data.select_dtypes(include=['object']).columns
        X_data = X_data.drop(columns=object_columns)

    y_data = uci_download.data.targets
    # select target for datasets with 2 targets
    if uci_id == 189:
        y_data = y_data["motor_UPDRS"]
    if uci_id == 713:
        y_data = y_data["verification.time"]

    y = y_data.squeeze()

    X = torch.tensor(X_data.values, dtype=torch.float32)
    y = torch.tensor(y.values, dtype=torch.float32)

    train_n = int(floor(0.8 * len(X)))
    train_x = X[:train_n, :].contiguous()
    train_y = y[:train_n].contiguous()

    test_x = X[train_n:, :].contiguous()
    test_y = y[train_n:].contiguous()

    # Create TensorDataset and DataLoader for training and test sets
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(test_x, test_y)

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    return train_x, train_y, test_x, test_y, train_loader, test_loader
    
def whole_process(train_x, train_y, test_x, test_y, train_loader, test_loader):

    data_dim = train_x.size(-1)

    class LargeFeatureExtractor(torch.nn.Sequential):
        def __init__(self):
            super(LargeFeatureExtractor, self).__init__()
            self.add_module('linear1', torch.nn.Linear(data_dim, 1000))
            self.add_module('relu1', torch.nn.ReLU())
            self.add_module('linear2', torch.nn.Linear(1000, 500))
            self.add_module('relu2', torch.nn.ReLU())
            self.add_module('linear3', torch.nn.Linear(500, 50))
            self.add_module('relu3', torch.nn.ReLU())
            self.add_module('linear4', torch.nn.Linear(50, 1))

    model = LargeFeatureExtractor()

    training_iterations = 100

    # Find optimal model hyperparameters
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam([
        {'params': model.parameters()}
    ], lr=0.01)


    def train():
        model.train()
        iterator = tqdm(range(training_iterations))
        for i in iterator:
            # Zero backprop gradients
            optimizer.zero_grad()
            # Get output from model
            output = model(train_x)
            # Calc loss and backprop derivatives
            loss = criterion(output, train_y)
            loss.backward()
            iterator.set_postfix(loss=loss.item())
            optimizer.step()

    start_time = time.time()
    %time train()
    end_time = time.time()

    wall_time = end_time - start_time

    model.eval()
    with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
        preds_test = model(test_x)

    test_rmse = torch.sqrt(torch.mean((preds_test - test_y) ** 2)).item()

    with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.settings.fast_pred_var():
        preds_train = model(train_x)

    train_rmse = torch.sqrt(torch.mean((preds_train - train_y) ** 2)).item()

    print('Test RMSE: {}'.format(test_rmse))
    print('Train RMSE: {}'.format(train_rmse))
    print('Wall time: {:.2f} seconds'.format(wall_time))

    return test_rmse, train_rmse, wall_time

experiment_datasets = [1, 275, 477, 189, 713, 186, 374, 183, 291, 294]

for experiment in experiment_datasets:
    print("Experiment " + str(experiment))

    trains = []
    tests = []
    times = []
    train_x, train_y, test_x, test_y, train_loader, test_loader = downloader(experiment)
    for run in range(3):
        # set seeds
        torch.manual_seed(run)
        print("run: " + str(run))

        test_rmse, train_rmse, wall_time = whole_process(train_x, train_y, test_x, test_y, train_loader, test_loader)
        trains.append(train_rmse)
        tests.append(test_rmse)
        times.append(wall_time)
    
    print("mean train performance= "+ str(np.mean(trains)))
    print("mean test performance= "+ str(np.mean(tests)))
    print("mean wall time= "+ str(np.mean(times)))

    print("std train= "+ str(np.std(trains, ddof=1)))
    print("std test= "+ str(np.std(tests, ddof=1)))



Experiment 1
run: 0


  0%|          | 0/100 [00:00<?, ?it/s]c:\Users\12lui\anaconda3\envs\NewTrading\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([3341])) that is different to the input size (torch.Size([3341, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
100%|██████████| 100/100 [00:12<00:00,  7.97it/s, loss=11.1]


CPU times: total: 1min 14s
Wall time: 12.6 s
Test RMSE: 2.7361459732055664
Train RMSE: 3.337834596633911
Wall time: 12.55 seconds
whh
run: 1


100%|██████████| 100/100 [00:12<00:00,  7.86it/s, loss=11.1]


CPU times: total: 1min 16s
Wall time: 12.7 s
Test RMSE: 2.7416114807128906
Train RMSE: 3.3371355533599854
Wall time: 12.72 seconds
whh
run: 2


...

CPU times: total: 3min 24s
Wall time: 34.2 s
Test RMSE: 17.535917282104492
Train RMSE: 17.659637451171875
Wall time: 34.24 seconds
whh
run: 1


100%|██████████| 100/100 [00:32<00:00,  3.08it/s, loss=311]   


CPU times: total: 3min 13s
Wall time: 32.4 s
Test RMSE: 17.520639419555664
Train RMSE: 17.632526397705078
Wall time: 32.42 seconds
whh
run: 2


100%|██████████| 100/100 [00:32<00:00,  3.06it/s, loss=308]   

CPU times: total: 3min 15s
Wall time: 32.7 s
Test RMSE: 17.458927154541016
Train RMSE: 17.55830192565918
Wall time: 32.71 seconds
whh
mean train performance= 17.61682192484538
mean test performance= 17.50516128540039
mean wall time= 33.12434752782186
std train= 0.05246136705790589
std test= 0.040762107566422276


In [3]:
trains

[17.659637451171875, 17.632526397705078, 17.55830192565918]